In [1]:
import json
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, Image, HTML
import random

In [2]:
base_path = Path("/nvme/VLMRobustness/LIDC")
answer_type = "CLOSED"
json_file_path = base_path / "validate.json"
output_file_path = base_path / f"rater_study_{answer_type.lower()}.json"

with open(json_file_path, 'r') as f:
    data = json.load(f)
#data = [element for element in data if element.get("q_lang")=="en"]
data = [element for element in data if element.get("answer_type")==answer_type]

# Initialize index and max index
current_index = 0
max_index = len(data)
selected_question_ids = []
selected_questions = []

def display_current_question():
    global current_index
    add_button.disabled = False
    remove_button.disabled = True
    element = data[current_index]
    if element["qid"] in selected_question_ids:
        add_button.disabled = True
        remove_button.disabled = False
    id_text.value = ""
    question_widget.value = f"<b>Question:</b> {element['question']}<br><b>Answer:</b> {element['answer']}<br><b>QID:</b> {element['qid']}"

def on_next_button_clicked(b):
    global current_index, max_index
    current_index = (current_index + 1) % max_index
    display_current_question()

def on_previous_button_clicked(b):
    global current_index, max_index
    if current_index == 0:
        return
    current_index = (current_index - 1) % max_index
    display_current_question()

def on_random_button_clicked(b):
    global current_index, max_index
    current_index = random.randint(0, max_index)
    display_current_question()

def on_add_button_clicked(b):
    global current_index, max_index, selected_questions, selected_question_ids
    element = data[current_index]
    selected_question_ids.append(element["qid"])
    selected_questions.append(element)
    add_button.disabled = True
    remove_button.diabled = False
    display_selected_questions()

def on_remove_button_clicked(b):
    global current_index, max_index, selected_questions, selected_question_ids
    element = data[current_index]
    selected_question_ids.remove(element["qid"])
    selected_questions = [e for e in selected_questions if e["qid"]!=element["qid"]]
    add_button.disabled = False
    remove_button.disabled = True
    display_selected_questions()

def on_id_button_clicked(b):
    global current_index, max_index
    question = [(index, element) for index, element in enumerate(data) if str(element["qid"]) == id_text.value]
    if len(question) == 1:
        current_index = question[0][0]
        display_current_question()
    else:
        print(f"couldn't find question {id_text.value}")

def on_save_clicked(b):
    with open(output_file_path, 'w') as f:
        data = json.dump(selected_questions, f, indent=4)
    print("saved")


question_widget = widgets.HTML()
previous_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")
random_button = widgets.Button(description="Random Sample")
add_button = widgets.Button(description="Add Sample to Eval")
remove_button = widgets.Button(description="Remove Sample from Eval", enabled=False)
id_text = widgets.Text(
    value='',
    placeholder='',
    disabled=False
)
id_button = widgets.Button(description="Jump to ID")
save_button = widgets.Button(description="Save questions to file")

next_button.on_click(on_next_button_clicked)
previous_button.on_click(on_previous_button_clicked)
random_button.on_click(on_random_button_clicked)
add_button.on_click(on_add_button_clicked)
remove_button.on_click(on_remove_button_clicked)
id_button.on_click(on_id_button_clicked)
save_button.on_click(on_save_clicked)

# Display the first image and information
display_current_question()

# Layout the widgets
buttons_previous_next_random = widgets.HBox([previous_button, next_button, random_button])
select_id = widgets.HBox([id_text, id_button])
add_remove_button = widgets.HBox([add_button, remove_button])
ui = widgets.VBox([question_widget, buttons_previous_next_random, select_id, add_remove_button, save_button])
display(ui)

selected_questions_widget = widgets.HTML()

def display_selected_questions():
    selected_questions_widget.value = f"================<br><b>SELECTED QUESTIONS (total {len(selected_questions)})</b><br>" +  "<br>================<br>".join(f"<b>Question:</b> {element['question']}<br><b>Answer:</b> {element['answer']}<br><b>QID:</b> {element['qid']}" for element in selected_questions)

display(selected_questions_widget)

HTML(value='')

saved
